In [2]:
import pandas as pd
import os

In [4]:
filenames = ["../generated/sources_rates.hdf5",
             "../generated/pulishers_rates.hdf5",
             "../generated/topics_rates.hdf5",
             "../generated/ent_rates.hdf5",
             "../generated/cat_rates.hdf5"]

In [2]:
pd.read_csv("../download/page_views.csv", nrows=10).head()

,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,1fd5f051fba643,120,31905835,1,RS,2
1,8557aa9004be3b,120,32053104,1,VN>44,2
2,c351b277a358f0,120,54013023,1,KR>12,1
3,8205775c5387f9,120,44196592,1,IN>16,2
4,9cb0ccd8458371,120,65817371,1,US>CA>807,2


## Calculating Contetnt popularity

### Load page_view grouped 
As needed for calculation of popularity (#viewed) for content: meta, topics, categories and entities

In [ ]:
filename = "../generated/page_view_grouped_for_meta.hdf5"

In [ ]:
def group_page_view(filename, chunksize= 10 * 10 ** 6):
    try:
        os.remove(filename)
    except OSError:
        pass

    loop = 0
    len_grouped = 0
    
    iterator = pd.read_csv("../download/page_views.csv", chunksize=chunksize, usecols=['document_id', 'platform', 'geo_location', 'traffic_source'], dtype={'document_id':int, 'platform':int, 'geo_location':str, 'traffic_source':int})
    
    for chunk in iterator:
        chunk.fillna({'geo_location':'na'},inplace=True)
    
        group = pd.DataFrame({'viewed':chunk.groupby(['document_id', 'platform', 'geo_location', 'traffic_source']).size()}).reset_index()
        group.to_hdf(filename, key='default', mode='a', append=True, complevel=9, complib='zlib')

        loop += 1
        len_grouped += len(group)
    
        print loop, len_grouped
        
    return len_grouped 

In [ ]:
def group_page_view_next(filename, prev_filename='', chunksize= 10 * 10 ** 6):
    try:
        os.remove(filename)
    except OSError:
        pass

    loop = 0
    len_grouped = 0
    
    iterator = pd.read_hdf(prev_filename, chunksize=chunksize)
    
    for chunk in iterator:
        group = pd.DataFrame({'viewed':chunk.groupby(['document_id', 'platform', 'geo_location', 'traffic_source'])['viewed'].sum()}).reset_index()
        group.to_hdf(filename, key='default', mode='a', append=True, complevel=9, complib='zlib')

        loop += 1
        len_grouped += len(group)
    
        print loop, len_grouped
        
    return len_grouped  

In [ ]:
i = 0
print 'Step '+str(i)
current_len = group_page_view(filename+'.'+str(i))
print current_len

In [ ]:
i = 0
current_len = 586781833
pd.read_hdf(filename+'.'+str(i), stop = 10).head()

In [ ]:
i+=1
next_len = group_page_view_next(filename = filename+'.'+str(i), prev_filename = filename+'.0')
print next_len

In [ ]:
586684201 - current_len

### Load content tables

In [ ]:
docs_meta = pd.read_csv("../download/documents_meta.csv", usecols=['document_id', 'source_id', 'publisher_id']).fillna(-1)
docs_meta.count()

In [ ]:
docs_topics = pd.read_csv("../download/documents_topics.csv")
docs_topics.count()

In [ ]:
docs_ent = pd.read_csv("../download/documents_entities.csv")
docs_ent.count()

In [ ]:
docs_cat = pd.read_csv("../download/documents_categories.csv")
docs_cat.count()

In [ ]:
pd.read_hdf("../generated/page_view_grouped_for_meta.hdf5.1", stop = 5).head()

### Initial joins

In [ ]:
def page_view_grouped_join_meta(input_filename, filenames, chunksize = 10 ** 6):
    for fn in filenames:
        try:
            os.remove(fn)
        except OSError:
            pass

    loop = 0
    len_source, len_publisher, let_topics, len_ent, len_cat = 0, 0, 0, 0, 0
    
    for chunk in pd.read_hdf(input_filename, chunksize=chunksize):
        chunk_meta = pd.merge(chunk, docs_meta, on='document_id')
        chunk_meta.drop(['document_id'], axis=1, inplace=True)
        chunk_source = pd.DataFrame({'source_viewed':chunk_meta.groupby(['source_id', 'platform', 'geo_location', 'traffic_source'])['viewed'].sum()}).reset_index()
        chunk_publisher = pd.DataFrame({'publisher_viewed':chunk_meta.groupby(['publisher_id', 'platform', 'geo_location', 'traffic_source'])['viewed'].sum()}).reset_index()
        del chunk_meta
        len_source += len(chunk_source)
        chunk_source.to_hdf(filenames[0], key='default', mode='a', append=True, complevel=9, complib='zlib')
        del chunk_source
        len_publisher += len(chunk_publisher)
        chunk_publisher.to_hdf(filenames[1], key='default', mode='a', append=True, complevel=9, complib='zlib')
        del chunk_publisher
        
        chunk_topics = pd.merge(chunk, docs_topics, on='document_id')
        chunk_topics['topic_viewed'] = chunk_topics['viewed']*chunk_topics['confidence_level']
        chunk_topics.drop(['document_id', 'viewed', 'confidence_level'], axis=1, inplace=True)
        chunk_topics = pd.DataFrame({'topic_viewed':chunk_topics.groupby(['topic_id', 'platform', 'geo_location', 'traffic_source'])['topic_viewed'].sum()}).reset_index()
        let_topics += len(chunk_topics)
        chunk_topics.to_hdf(filenames[2], key='default', mode='a', append=True, complevel=9, complib='zlib')
        del chunk_topics
        
        chunk_ent = pd.merge(chunk, docs_ent, on='document_id')
        chunk_ent['ent_viewed'] = chunk_ent['viewed']*chunk_ent['confidence_level']
        chunk_ent.drop(['document_id', 'viewed', 'confidence_level'], axis=1, inplace=True)
        chunk_ent = pd.DataFrame({'ent_viewed':chunk_ent.groupby(['entity_id', 'platform', 'geo_location', 'traffic_source'])['ent_viewed'].sum()}).reset_index()
        len_ent += len(chunk_ent)
        chunk_ent.to_hdf(filenames[3], key='default', mode='a', append=True, complevel=9, complib='zlib')
        del chunk_ent
        
        chunk_cat = pd.merge(chunk, docs_cat, on='document_id')
        chunk_cat['cat_viewed'] = chunk_cat['viewed']*chunk_cat['confidence_level']
        chunk_cat.drop(['document_id', 'viewed', 'confidence_level'], axis=1, inplace=True)
        chunk_cat = pd.DataFrame({'cat_viewed':chunk_cat.groupby(['category_id', 'platform', 'geo_location', 'traffic_source'])['cat_viewed'].sum()}).reset_index()
        len_cat += len(chunk_cat)
        chunk_cat.to_hdf(filenames[4], key='default', mode='a', append=True, complevel=9, complib='zlib')
        del chunk_cat
        
        loop += 1
        print loop, len_source, len_publisher, let_topics, len_ent, len_cat
        
    return len_source, len_publisher, let_topics, len_ent, len_cat   

In [ ]:
i = 0
print 'Step '+str(i)
print page_view_grouped_join_meta("../generated/page_view_grouped_for_meta.hdf5.1", filenames)

### Compacting new Content tables

In [ ]:
def group_content(filen, measure, dims, chunksize, step, ftype='hdf', suffix=''):
    
    if step == 0:
        prev = filenames[filen] + suffix
    else:
        prev = filenames[filen] + suffix+'.'+str(step-1)
        
    output_filename = filenames[filen]+suffix+'.'+str(step)    

    try:
        os.remove(output_filename)
    except OSError:
        pass

    len_total = 0
    i = 0
    
    iterator = pd.read_hdf(prev, chunksize = chunksize) if ftype=='hdf' else pd.read_csv(prev, chunksize = chunksize)
    
    for chunk in iterator:
        group = pd.DataFrame({measure: chunk.groupby(dims)[measure].sum()}).reset_index()
                                                          
        group.to_hdf(output_filename, key='default', mode='a', append=True, complevel=9, complib='zlib')                                                      
                                                          
        len_total += len(group)
        i+=1
        print i, len_total 
    
    print 'Done'

#### Encoding geo_location (also cuts records with other locations not used in events)

In [3]:
geo_locations = pd.read_csv("../generated/geo_locations.csv", dtype={'geo_location':str})
geo_locations.head()

,geo,geo_location
0,-1,--
1,0,A1
2,1,A2
3,2,AD
4,3,AD>02


In [4]:
geo_locations.count()

geo             2987
geo_location    2987
dtype: int64

In [ ]:
def encode_geo_location(filename, next_filename, geo_locations, ftype='hdf'):
    data = pd.read_hdf(filename)
    index = (data['geo_location'] == '-->-->0') | (data['geo_location'] == 'na')
    data.loc[index,'geo_location'] = '--'
    
    data = data.merge(geo_locations, on='geo_location')
    data.drop('geo_location', axis=1, inplace=True)
    
    if ftype=='hdf':
        data.to_hdf(next_filename, key='default', mode='w', complevel=5, complib='zlib')
    else:    
        data.to_csv(next_filename, index=False)
        
    return len(data), sum(data['geo'] == -1)

#### Sources

In [5]:
measure = 'source_viewed'
dims = [u'source_id', u'platform', u'geo_location', u'traffic_source']
#group_content(0, measure, dims, chunksize = 10 ** 7, step=0)  #~104 *10e6
#group_content(0, measure, dims, chunksize = 10 ** 7, step=1)# 64,502,530
#group_content(0, measure, dims, chunksize = 2 * 10 ** 7, step=2) # 36,038,451
#group_content(0, measure, dims, chunksize = 3 * 10 ** 7, step=3) # ~21,000,000
#group_content(0, measure, dims, chunksize = 3 * 10 ** 7, step=4) # 15,268,303
#group_content(0, measure, dims, chunksize = 2 * 10 ** 7, step=5) # 15,268,303  FINAL
pd.read_hdf(filenames[0]+'.5', stop = 10).head()

,source_id,platform,geo_location,traffic_source,source_viewed
0,-1.0,1,--,1,30
1,-1.0,1,--,2,20
2,-1.0,1,--,3,1
3,-1.0,1,-->-->0,1,2
4,-1.0,1,A1,1,10


In [ ]:
print encode_geo_location(filenames[0]+'.5', filenames[0]+'.csv.6', geo_locations, ftype='csv')  #(15231938, 15150)

In [ ]:
pd.read_csv(filenames[0]+'.csv.6', nrows = 10).head()

In [ ]:
dims = [u'source_id', u'platform', u'traffic_source', u'geo']
print dims, measure

In [ ]:
group_content(0, measure, dims, chunksize = 2 * 10 ** 7, step=7, ftype='csv', suffix='.csv') #15,225,546

In [6]:
pd.read_hdf(filenames[0]+'.csv.7', stop = 10).head()

,source_id,platform,traffic_source,geo,source_viewed
0,-1.0,1,1,-1,32
1,-1.0,1,1,0,10
2,-1.0,1,1,11,13
3,-1.0,1,1,13,49
4,-1.0,1,1,15,1


#### Publishers

In [ ]:
measure = 'publisher_viewed'
dims = [u'publisher_id', u'platform', u'geo_location', u'traffic_source']
#group_content(1, measure, dims, chunksize = 2 * 10 ** 7, step=0) # 24,880,740
#group_content(1, measure, dims, chunksize = 2 * 10 ** 7, step=1) #7,745,188
#group_content(1, measure, dims, chunksize = 10 ** 7, step=2) #4,967,426
#group_content(1, measure, dims, chunksize = 10 ** 7, step=3) #4,967,426  FINAL
pd.read_hdf(filenames[1]+'.3', stop = 10).head()

In [ ]:
print encode_geo_location(filenames[1]+'.3', filenames[1]+'.csv.4', geo_locations, ftype='csv') #(4,938,588, 6188)

In [ ]:
dims = [u'publisher_id', u'platform', u'traffic_source', u'geo']

In [ ]:
group_content(1, measure, dims, chunksize = 10 ** 7, step=5, ftype='csv', suffix='.csv') #4,935,288

In [7]:
pd.read_hdf(filenames[1]+'.csv.5', stop = 10).head()

,publisher_id,platform,traffic_source,geo,publisher_viewed
0,-1.0,1,1,-1,22
1,-1.0,1,1,0,8
2,-1.0,1,1,11,12
3,-1.0,1,1,13,33
4,-1.0,1,1,15,1


#### Topics

In [ ]:
measure = u'topic_viewed'
dims = [u'topic_id', u'platform', u'geo_location', u'traffic_source']
#group_content(2, measure, dims, chunksize = 5* 10 ** 6, step=0)  # 206,947,198
#group_content(2, measure, dims, chunksize = 10 ** 7, step=1)  # 69,690,010
#group_content(2, measure, dims, chunksize = 10 ** 7, step=2)  # 29,792,622
#group_content(2, measure, dims, chunksize = 10 ** 7, step=3)  # 14,919,647
#group_content(2, measure, dims, chunksize = 10 ** 7, step=4)  # 10,508,890
#group_content(2, measure, dims, chunksize = 10 ** 7, step=5)  # 6,383,228
#group_content(2, measure, dims, chunksize = 10 ** 7, step=6)  # 5,925,270
#group_content(2, measure, dims, chunksize = 10 ** 7, step=7)  # 5,925,270  FINAL
pd.read_hdf(filenames[2]+'.7', stop = 10).head()

In [ ]:
print encode_geo_location(filenames[2]+'.7', filenames[2]+'.csv.8', geo_locations, ftype='csv') #(5,822,091, 7612)

In [ ]:
dims = [u'topic_id', u'platform', u'traffic_source', u'geo']

In [ ]:
group_content(2, measure, dims, chunksize = 10 ** 7, step=9, ftype='csv', suffix='.csv')

In [5]:
pd.read_hdf(filenames[2]+'.csv.9', stop = 10).head()

,topic_id,platform,traffic_source,geo,topic_viewed
0,0,1,1,-1,20.310514
1,0,1,1,0,38.536388
2,0,1,1,1,4.740327
3,0,1,1,2,0.186612
4,0,1,1,4,0.037480


#### Entities

In [5]:
pd.read_hdf(filenames[3], stop = 10).head()

,entity_id,platform,geo_location,traffic_source,ent_viewed
0,000228e7d9e9975f14e5cc1aeebab6a6,1,US>CT>533,2,0.853479
1,000228e7d9e9975f14e5cc1aeebab6a6,1,US>WA>819,2,0.853479
2,000228e7d9e9975f14e5cc1aeebab6a6,2,US>IL>602,2,0.853479
3,000228e7d9e9975f14e5cc1aeebab6a6,2,US>NY>538,2,0.853479
4,0003a98bea82feeafa038d1b7e191bfe,1,AT>06,2,0.295132


#### Categories

In [ ]:
measure = u'cat_viewed'
dims = [u'category_id', u'platform', u'geo_location', u'traffic_source']
#group_content(4, measure, dims, chunksize = 10 ** 7, step=0)  #19,530,552
#group_content(4, measure, dims, chunksize = 10 ** 7, step=1)  #3,253,069
#group_content(4, measure, dims, chunksize = 10 ** 7, step=2)  #1,792,992
#group_content(4, measure, dims, chunksize = 10 ** 7, step=3)  #1,792,992  FINAL
pd.read_hdf(filenames[4]+'.3', stop = 10).head()

In [ ]:
print encode_geo_location(filenames[4]+'.3', filenames[4]+'.csv.4', geo_locations, ftype='csv') #(1,755,953, 2222)

In [ ]:
dims = [u'category_id', u'platform', u'traffic_source', u'geo']

In [ ]:
group_content(4, measure, dims, chunksize = 10 ** 7, step=5, ftype='csv', suffix='.csv') #1,754,514

In [9]:
pd.read_hdf(filenames[4]+'.csv.5', stop = 10).head()

,category_id,platform,traffic_source,geo,cat_viewed
0,1000,1,1,-1,20.822754
1,1000,1,1,0,239.240352
2,1000,1,1,1,3.141090
3,1000,1,1,5,0.032317
4,1000,1,1,8,0.128752
